In [19]:
import requests
import json
from bs4 import BeautifulSoup
import m3u8
import subprocess

In [20]:
SNAP_URL = 'https://t.snapchat.com/BAxCQe9u'
OUTPUT_NAME = 'mysnap'
chunk_size = 256

In [21]:
r = requests.get(SNAP_URL)

In [22]:
soup = BeautifulSoup(r.text, 'html.parser')

In [23]:
script_tag = soup.find("script", {"id": "__NEXT_DATA__"})

In [24]:
snap_data = json.loads(script_tag.string)

In [25]:
master_m3u8_url = snap_data['props']['pageProps']['preselectedStory']['premiumStory']['playerStory']['snapList'][0]['snapUrls']['mediaUrl']

In [26]:
r = requests.get(master_m3u8_url)

In [27]:
master_m3u8 = m3u8.loads(r.text)

In [28]:
video_m3u8_url = 'https://cf-st.sc-cdn.net/d/' + master_m3u8.data['playlists'][-1]['uri']

In [29]:
audio_m3u8_url = 'https://cf-st.sc-cdn.net/d/' + master_m3u8.data['media'][-1]['uri']

In [30]:
rv = requests.get(video_m3u8_url)
ra = requests.get(audio_m3u8_url)

In [31]:
video_m3u8 = m3u8.loads(rv.text)
audio_m3u8 = m3u8.loads(ra.text)

In [32]:
video_url = 'https://cf-st.sc-cdn.net/d/' + video_m3u8.data['segment_map'][0]['uri']
audio_url = 'https://cf-st.sc-cdn.net/d/' + audio_m3u8.data['segment_map'][0]['uri']

In [33]:
video_url

'https://cf-st.sc-cdn.net/d/TXQJ8MboZ0Uc7miSwkpDS.85.mp4'

In [34]:
r = requests.get(url=video_url, stream=True)
with open(f"{OUTPUT_NAME}-video.mp4", "wb") as f:
    for chunk in r.iter_content(chunk_size=chunk_size):
        f.write(chunk)

In [35]:
r = requests.get(url=audio_url, stream=True)
with open(f"{OUTPUT_NAME}-audio.mp4", "wb") as f:
    for chunk in r.iter_content(chunk_size=chunk_size):
        f.write(chunk)

In [36]:
subprocess.run(['ffmpeg', '-i', f"{OUTPUT_NAME}-video.mp4", '-i', f"{OUTPUT_NAME}-audio.mp4", '-c:v', 'copy', '-c:a', 'aac', f"{OUTPUT_NAME}.mp4"])

ffmpeg version n6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.1.1 (GCC) 20230429
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable

CompletedProcess(args=['ffmpeg', '-i', 'mysnap-video.mp4', '-i', 'mysnap-audio.mp4', '-c:v', 'copy', '-c:a', 'aac', 'mysnap.mp4'], returncode=0)